In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/navy/

/content/drive/MyDrive/navy


# Naval Mine Warfare Simulator

### 3D tactical mine field simulation and risk analysis for naval operations.

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from matplotlib.patches import Circle
import matplotlib.patches as mpatches
from dataclasses import dataclass
from typing import List, Tuple, Dict, Literal
import json
import os
from datetime import datetime
from enum import Enum

# 고품질 시각화 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = '#f8f9fa'
plt.rcParams['grid.alpha'] = 0.3

class ThreatLevel(Enum):
    """기뢰 수준"""
    MODERATE = (73, 0.50)  # 기뢰 150개, 목표 위험률 50%
    HIGH = (145, 0.75)      # 기뢰 300개, 목표 위험률 75%
    CRITICAL = (241, 0.90)  # 기뢰 450개, 목표 위험률 90%

class RouteScenario(Enum):
    """경로 시나리오"""
    DIRECT = "Straight"
    ZIGZAG = "Zig-zag"
    DEEP_DIVE = "Deep Dive"
    COASTAL = "Coastal Route"

@dataclass
class TacticalMineConfig:
    """전술적 기뢰 부설 설정"""
    area_width: float = 8000
    area_height: float = 8000
    max_depth: float = 250

    # 위협 수준
    threat_level: ThreatLevel = ThreatLevel.HIGH

    # 기뢰 배치 전략
    core_route_width: float = 1000  # 핵심 항로 폭 (m)
    linear_density: float = 0.7  # 선형 배치 비율 (0-1)
    random_density: float = 0.3  # 랜덤 배치 비율 (0-1)

    # 부유 기뢰 (수면, 3-50m)
    surface_mine_depth_range: Tuple[float, float] = (3, 50)
    surface_mine_spacing: Tuple[float, float] = (70, 150)

    # 계류/침저 기뢰 (30-55m 수심, 간격 30-55m)
    subsurface_mine_depth_range: Tuple[float, float] = (30, 55)
    subsurface_mine_spacing: Tuple[float, float] = (30, 55)

    # 기뢰 파라미터
    mine_radius: float = 32

    # 닻자망
    num_nets: int = 30
    net_width: float = 50
    net_length_range: Tuple[float, float] = (300, 1000)
    net_depth_range: Tuple[float, float] = (50, 200)

    # 함정 파라미터
    vessel_width: float = 20
    vessel_draft: float = 8
    submarine_width: float = 12

    # 시뮬레이션
    num_simulations: int = 250
    path_sampling_points: int = 200

class SurfaceMine:
    """부유 기뢰"""
    def __init__(self, x: float, y: float, z: float, radius: float, placement_type: str = "random"):
        self.x = x
        self.y = y
        self.z = z
        self.radius = radius
        self.type = "surface"
        self.placement_type = placement_type

    def check_collision_2d(self, path_points: np.ndarray, vessel_width: float,
                          vessel_draft: float) -> bool:
        if vessel_draft < self.z:
            return False
        distances = np.sqrt((path_points[:, 0] - self.x)**2 +
                          (path_points[:, 1] - self.y)**2)
        return np.any(distances <= (self.radius + vessel_width/2))

    def check_collision_3d(self, path_points: np.ndarray, vessel_width: float) -> bool:
        surface_points = path_points[path_points[:, 2] <= self.z + 20]
        if len(surface_points) == 0:
            return False
        distances = np.sqrt((surface_points[:, 0] - self.x)**2 +
                          (surface_points[:, 1] - self.y)**2)
        return np.any(distances <= (self.radius + vessel_width/2))

class MooredMine:
    """계류 기뢰"""
    def __init__(self, x: float, y: float, z: float, radius: float, placement_type: str = "random"):
        self.x = x
        self.y = y
        self.z = z
        self.radius = radius
        self.type = "moored"
        self.placement_type = placement_type

    def check_collision_3d(self, path_points: np.ndarray, vessel_width: float) -> bool:
        distances = np.sqrt((path_points[:, 0] - self.x)**2 +
                          (path_points[:, 1] - self.y)**2 +
                          (path_points[:, 2] - self.z)**2)
        return np.any(distances <= (self.radius + vessel_width/2))

class BottomMine:
    """침저 기뢰"""
    def __init__(self, x: float, y: float, z_bottom: float, radius: float, placement_type: str = "random"):
        self.x = x
        self.y = y
        self.z = z_bottom
        self.radius = radius
        self.type = "bottom"
        self.placement_type = placement_type

    def check_collision_3d(self, path_points: np.ndarray, vessel_width: float) -> bool:
        distances = np.sqrt((path_points[:, 0] - self.x)**2 +
                          (path_points[:, 1] - self.y)**2 +
                          (path_points[:, 2] - self.z)**2)
        return np.any(distances <= (self.radius + vessel_width/2))

class Net3D:
    """닻자망"""
    def __init__(self, x1: float, y1: float, x2: float, y2: float,
                 depth_top: float, depth_bottom: float, width: float):
        self.x1, self.y1 = x1, y1
        self.x2, self.y2 = x2, y2
        self.z_top = depth_top
        self.z_bottom = depth_bottom
        self.width = width
        self.length = np.sqrt((x2-x1)**2 + (y2-y1)**2)
        self.type = "net"

    def check_collision_2d(self, path_points: np.ndarray, vessel_width: float,
                          vessel_draft: float) -> bool:
        if vessel_draft < self.z_top:
            return False

        total_width = (self.width + vessel_width) / 2

        for i in range(len(path_points) - 1):
            p1 = path_points[i, :2]
            p2 = path_points[i + 1, :2]

            if self._segment_intersects_2d(p1, p2, total_width):
                return True

        return False

    def check_collision_3d(self, path_points: np.ndarray, vessel_width: float) -> bool:
        total_width = (self.width + vessel_width) / 2

        for i in range(len(path_points) - 1):
            p1 = path_points[i]
            p2 = path_points[i + 1]

            if not (self.z_top <= p1[2] <= self.z_bottom or
                   self.z_top <= p2[2] <= self.z_bottom):
                continue

            if self._segment_intersects_2d(p1[:2], p2[:2], total_width):
                return True

        return False

    def _segment_intersects_2d(self, p1: np.ndarray, p2: np.ndarray,
                              safe_distance: float) -> bool:
        q1 = np.array([self.x1, self.y1])
        q2 = np.array([self.x2, self.y2])

        dist1 = self._point_to_segment_distance(p1, q1, q2)
        dist2 = self._point_to_segment_distance(p2, q1, q2)
        dist3 = self._point_to_segment_distance(q1, p1, p2)
        dist4 = self._point_to_segment_distance(q2, p1, p2)

        return min(dist1, dist2, dist3, dist4) <= safe_distance

    def _point_to_segment_distance(self, point: np.ndarray,
                                   seg_start: np.ndarray, seg_end: np.ndarray) -> float:
        segment = seg_end - seg_start
        point_vec = point - seg_start

        segment_length_sq = np.dot(segment, segment)

        if segment_length_sq < 1e-6:
            return np.linalg.norm(point - seg_start)

        t = np.clip(np.dot(point_vec, segment) / segment_length_sq, 0, 1)
        projection = seg_start + t * segment

        return np.linalg.norm(point - projection)

class TacticalMineSimulation:
    """전술적 기뢰 부설 시뮬레이션"""

    def __init__(self, config: TacticalMineConfig):
        self.config = config
        self.surface_mines: List[SurfaceMine] = []
        self.moored_mines: List[MooredMine] = []
        self.bottom_mines: List[BottomMine] = []
        self.nets: List[Net3D] = []

        self.results = {
            'surface_vessel': {'mine_hits': 0, 'net_hits': 0, 'both_hits': 0, 'safe': 0},
            'submarine': {'mine_hits': 0, 'net_hits': 0, 'both_hits': 0, 'safe': 0}
        }

        # 시나리오별 결과 저장
        self.scenario_results = {}

    def _calculate_core_route(self, start: Tuple[float, float],
                             end: Tuple[float, float]) -> Tuple[np.ndarray, np.ndarray]:
        """핵심 항로 중심선과 수직 벡터 계산"""
        direction = np.array([end[0] - start[0], end[1] - start[1]])
        direction = direction / np.linalg.norm(direction)
        perpendicular = np.array([-direction[1], direction[0]])
        return direction, perpendicular

    def generate_tactical_mines(self, start: Tuple[float, float],
                                end: Tuple[float, float],
                                seed: int = None):
        """전술적 기뢰 부설"""
        if seed is not None:
            np.random.seed(seed)

        total_mines = self.config.threat_level.value[0]

        # 기뢰 종류별 비율 (부유:계류:침저 = 3:4:3)
        num_surface = int(total_mines * 0.3)
        num_moored = int(total_mines * 0.4)
        num_bottom = total_mines - num_surface - num_moored

        # 핵심 항로 정보
        direction, perpendicular = self._calculate_core_route(start, end)
        route_length = np.sqrt((end[0]-start[0])**2 + (end[1]-start[1])**2)

        # === 부유 기뢰 배치 ===
        self.surface_mines = []
        num_linear_surface = int(num_surface * self.config.linear_density)
        spacing = np.random.uniform(*self.config.surface_mine_spacing)
        num_line_mines = int(route_length / spacing)

        for i in range(min(num_linear_surface, num_line_mines)):
            t = i / max(num_line_mines - 1, 1)
            base_x = start[0] + t * (end[0] - start[0])
            base_y = start[1] + t * (end[1] - start[1])

            offset = np.random.uniform(-self.config.core_route_width/3,
                                      self.config.core_route_width/3)
            x = base_x + offset * perpendicular[0]
            y = base_y + offset * perpendicular[1]
            z = np.random.uniform(*self.config.surface_mine_depth_range)

            x = np.clip(x, 0, self.config.area_width)
            y = np.clip(y, 0, self.config.area_height)

            self.surface_mines.append(SurfaceMine(x, y, z, self.config.mine_radius, "linear"))

        # 랜덤 배치
        num_random_surface = num_surface - len(self.surface_mines)
        for _ in range(num_random_surface):
            mid_x = (start[0] + end[0]) / 2
            mid_y = (start[1] + end[1]) / 2

            x = np.random.normal(mid_x, self.config.area_width / 4)
            y = np.random.normal(mid_y, self.config.area_height / 4)
            z = np.random.uniform(*self.config.surface_mine_depth_range)

            x = np.clip(x, 0, self.config.area_width)
            y = np.clip(y, 0, self.config.area_height)

            self.surface_mines.append(SurfaceMine(x, y, z, self.config.mine_radius, "random"))

        # === 계류 기뢰 배치 ===
        self.moored_mines = []
        num_linear_moored = int(num_moored * self.config.linear_density)
        spacing = np.random.uniform(*self.config.subsurface_mine_spacing)
        num_line_mines = int(route_length / spacing)

        for i in range(min(num_linear_moored, num_line_mines)):
            t = i / max(num_line_mines - 1, 1)
            base_x = start[0] + t * (end[0] - start[0])
            base_y = start[1] + t * (end[1] - start[1])

            offset = np.random.uniform(-self.config.core_route_width/2,
                                      self.config.core_route_width/2)
            x = base_x + offset * perpendicular[0]
            y = base_y + offset * perpendicular[1]
            z = np.random.uniform(*self.config.subsurface_mine_depth_range)

            x = np.clip(x, 0, self.config.area_width)
            y = np.clip(y, 0, self.config.area_height)

            self.moored_mines.append(MooredMine(x, y, z, self.config.mine_radius, "linear"))

        num_random_moored = num_moored - len(self.moored_mines)
        for _ in range(num_random_moored):
            mid_x = (start[0] + end[0]) / 2
            mid_y = (start[1] + end[1]) / 2

            x = np.random.normal(mid_x, self.config.area_width / 3)
            y = np.random.normal(mid_y, self.config.area_height / 3)
            z = np.random.uniform(*self.config.subsurface_mine_depth_range)

            x = np.clip(x, 0, self.config.area_width)
            y = np.clip(y, 0, self.config.area_height)

            self.moored_mines.append(MooredMine(x, y, z, self.config.mine_radius, "random"))

        # === 침저 기뢰 배치 ===
        self.bottom_mines = []
        num_linear_bottom = int(num_bottom * self.config.linear_density)
        spacing = np.random.uniform(*self.config.subsurface_mine_spacing)
        num_line_mines = int(route_length / spacing)

        for i in range(min(num_linear_bottom, num_line_mines)):
            t = i / max(num_line_mines - 1, 1)
            base_x = start[0] + t * (end[0] - start[0])
            base_y = start[1] + t * (end[1] - start[1])

            offset = np.random.uniform(-self.config.core_route_width/2,
                                      self.config.core_route_width/2)
            x = base_x + offset * perpendicular[0]
            y = base_y + offset * perpendicular[1]

            x = np.clip(x, 0, self.config.area_width)
            y = np.clip(y, 0, self.config.area_height)

            self.bottom_mines.append(BottomMine(x, y, self.config.max_depth,
                                               self.config.mine_radius, "linear"))

        num_random_bottom = num_bottom - len(self.bottom_mines)
        for _ in range(num_random_bottom):
            mid_x = (start[0] + end[0]) / 2
            mid_y = (start[1] + end[1]) / 2

            x = np.random.normal(mid_x, self.config.area_width / 3)
            y = np.random.normal(mid_y, self.config.area_height / 3)

            x = np.clip(x, 0, self.config.area_width)
            y = np.clip(y, 0, self.config.area_height)

            self.bottom_mines.append(BottomMine(x, y, self.config.max_depth,
                                               self.config.mine_radius, "random"))

        # === 닻자망 배치 ===
        self.nets = []
        for _ in range(self.config.num_nets):
            x1 = np.random.uniform(0, self.config.area_width)
            y1 = np.random.uniform(0, self.config.area_height)

            length = np.random.uniform(*self.config.net_length_range)
            angle = np.random.uniform(0, 2 * np.pi)

            x2 = np.clip(x1 + length * np.cos(angle), 0, self.config.area_width)
            y2 = np.clip(y1 + length * np.sin(angle), 0, self.config.area_height)

            z_top = np.random.uniform(*self.config.net_depth_range)
            z_bottom = z_top + np.random.uniform(50, 150)

            self.nets.append(Net3D(x1, y1, x2, y2, z_top, z_bottom, self.config.net_width))

    def generate_path_2d(self, start: Tuple[float, float],
                        end: Tuple[float, float]) -> np.ndarray:
        """수상함 2D 경로"""
        t = np.linspace(0, 1, self.config.path_sampling_points)
        x = start[0] + t * (end[0] - start[0])
        y = start[1] + t * (end[1] - start[1])
        z = np.zeros_like(x)
        return np.column_stack([x, y, z])

    def generate_path_3d(self, start: Tuple[float, float, float],
                        end: Tuple[float, float, float]) -> np.ndarray:
        """잠수함 3D 경로 (직선)"""
        t = np.linspace(0, 1, self.config.path_sampling_points)
        x = start[0] + t * (end[0] - start[0])
        y = start[1] + t * (end[1] - start[1])
        z = start[2] + t * (end[2] - start[2])
        return np.column_stack([x, y, z])

    def generate_scenario_path(self, scenario: RouteScenario,
                              start: Tuple[float, float, float],
                              end: Tuple[float, float, float]) -> np.ndarray:
        """시나리오별 경로 생성"""

        if scenario == RouteScenario.DIRECT:
            return self.generate_path_3d(start, end)

        elif scenario == RouteScenario.ZIGZAG:
            # 좌우 지그재그 경로
            waypoints = []
            num_zigs = 5
            for i in range(num_zigs):
                t = (i + 1) / (num_zigs + 1)
                x = start[0] + t * (end[0] - start[0])
                y = start[1] + t * (end[1] - start[1])
                offset = 800 * (-1 if i % 2 == 0 else 1)
                y += offset
                z = start[2] + t * (end[2] - start[2])
                waypoints.append((x, y, z))

            # 연결
            paths = [self.generate_path_3d(start, waypoints[0])]
            for i in range(len(waypoints) - 1):
                paths.append(self.generate_path_3d(waypoints[i], waypoints[i+1]))
            paths.append(self.generate_path_3d(waypoints[-1], end))

            return np.vstack(paths)

        elif scenario == RouteScenario.DEEP_DIVE:
            # 중간에 깊이 잠수
            mid_x = (start[0] + end[0]) / 2
            mid_y = (start[1] + end[1]) / 2
            mid_z = min(250, self.config.max_depth - 30)  # 안전 여유
            mid = (mid_x, mid_y, mid_z)

            path1 = self.generate_path_3d(start, mid)
            path2 = self.generate_path_3d(mid, end)
            return np.vstack([path1, path2])

        elif scenario == RouteScenario.COASTAL:
            # 연안 우회 (해역 가장자리)
            waypoint1 = (start[0] + 1000, 500, start[2])
            waypoint2 = (end[0] - 1000, 500, end[2])

            path1 = self.generate_path_3d(start, waypoint1)
            path2 = self.generate_path_3d(waypoint1, waypoint2)
            path3 = self.generate_path_3d(waypoint2, end)

            return np.vstack([path1, path2, path3])

    def check_surface_vessel_safety(self, path: np.ndarray) -> Tuple[bool, bool]:
        """수상함 안전성 검사"""
        mine_hit = any(mine.check_collision_2d(path, self.config.vessel_width,
                                               self.config.vessel_draft)
                      for mine in self.surface_mines)

        net_hit = any(net.check_collision_2d(path, self.config.vessel_width,
                                             self.config.vessel_draft)
                     for net in self.nets)

        return mine_hit, net_hit

    def check_submarine_safety(self, path: np.ndarray) -> Tuple[bool, bool]:
        """잠수함 안전성 검사"""
        mine_hit = False

        for mine in self.surface_mines:
            if mine.check_collision_3d(path, self.config.submarine_width):
                mine_hit = True
                break

        if not mine_hit:
            for mine in self.moored_mines:
                if mine.check_collision_3d(path, self.config.submarine_width):
                    mine_hit = True
                    break

        if not mine_hit:
            for mine in self.bottom_mines:
                if mine.check_collision_3d(path, self.config.submarine_width):
                    mine_hit = True
                    break

        net_hit = any(net.check_collision_3d(path, self.config.submarine_width)
                     for net in self.nets)

        return mine_hit, net_hit

    def run_simulation(self,
                      surface_start: Tuple[float, float],
                      surface_end: Tuple[float, float],
                      sub_start: Tuple[float, float, float],
                      sub_end: Tuple[float, float, float],
                      num_iterations: int = None,
                      verbose: bool = True) -> Dict:
        """시뮬레이션 실행"""
        if num_iterations is None:
            num_iterations = self.config.num_simulations

        self.results = {
            'surface_vessel': {'mine_hits': 0, 'net_hits': 0, 'both_hits': 0, 'safe': 0},
            'submarine': {'mine_hits': 0, 'net_hits': 0, 'both_hits': 0, 'safe': 0}
        }

        print_interval = max(1, num_iterations // 20)

        for i in range(num_iterations):
            self.generate_tactical_mines(surface_start, surface_end, seed=i)

            # 수상함
            surface_path = self.generate_path_2d(surface_start, surface_end)
            mine_hit, net_hit = self.check_surface_vessel_safety(surface_path)

            if mine_hit and net_hit:
                self.results['surface_vessel']['both_hits'] += 1
            elif mine_hit:
                self.results['surface_vessel']['mine_hits'] += 1
            elif net_hit:
                self.results['surface_vessel']['net_hits'] += 1
            else:
                self.results['surface_vessel']['safe'] += 1

            # 잠수함
            sub_path = self.generate_path_3d(sub_start, sub_end)
            mine_hit, net_hit = self.check_submarine_safety(sub_path)

            if mine_hit and net_hit:
                self.results['submarine']['both_hits'] += 1
            elif mine_hit:
                self.results['submarine']['mine_hits'] += 1
            elif net_hit:
                self.results['submarine']['net_hits'] += 1
            else:
                self.results['submarine']['safe'] += 1

            if verbose and (i + 1) % print_interval == 0:
                progress = (i + 1) / num_iterations * 100
                print(f"Progress: {progress:.1f}% ({i+1}/{num_iterations})")

        return self.calculate_statistics(num_iterations)

    def run_scenario_comparison(self,
                               sub_start: Tuple[float, float, float],
                               sub_end: Tuple[float, float, float],
                               num_iterations: int = 100) -> Dict:
        """여러 경로 시나리오 비교"""
        print("\n" + "="*80)
        print("ROUTE SCENARIO COMPARISON".center(80))
        print("="*80)

        scenarios = list(RouteScenario)
        scenario_results = {}

        for scenario in scenarios:
            print(f"\nTesting {scenario.value} scenario...")

            results = {'mine_hits': 0, 'net_hits': 0, 'both_hits': 0, 'safe': 0}

            for i in range(num_iterations):
                self.generate_tactical_mines((sub_start[0], sub_start[1]),
                                            (sub_end[0], sub_end[1]), seed=i)

                path = self.generate_scenario_path(scenario, sub_start, sub_end)
                mine_hit, net_hit = self.check_submarine_safety(path)

                if mine_hit and net_hit:
                    results['both_hits'] += 1
                elif mine_hit:
                    results['mine_hits'] += 1
                elif net_hit:
                    results['net_hits'] += 1
                else:
                    results['safe'] += 1

            # 통계 계산
            total = num_iterations
            scenario_results[scenario.value] = {
                'mine_hit_prob': results['mine_hits'] / total,
                'net_hit_prob': results['net_hits'] / total,
                'both_hit_prob': results['both_hits'] / total,
                'any_hit_prob': (results['mine_hits'] + results['net_hits'] +
                               results['both_hits']) / total,
                'safe_prob': results['safe'] / total,
                'counts': results
            }

            print(f"  Safe: {scenario_results[scenario.value]['safe_prob']*100:.1f}%, "
                  f"Risk: {scenario_results[scenario.value]['any_hit_prob']*100:.1f}%")

        self.scenario_results = scenario_results
        return scenario_results

    def calculate_statistics(self, total: int) -> Dict:
        """통계 계산"""
        stats = {}

        for vessel_type in ['surface_vessel', 'submarine']:
            r = self.results[vessel_type]
            stats[vessel_type] = {
                'mine_hit_prob': r['mine_hits'] / total,
                'net_hit_prob': r['net_hits'] / total,
                'both_hit_prob': r['both_hits'] / total,
                'any_hit_prob': (r['mine_hits'] + r['net_hits'] + r['both_hits']) / total,
                'safe_prob': r['safe'] / total,
                'counts': r
            }

        return stats

    def export_results_json(self, stats: Dict, threat_name: str,
                           output_dir: str, scenario_results: Dict = None):
        """JSON 결과 내보내기"""
        export_data = {
            'metadata': {
                'timestamp': datetime.now().isoformat(),
                'threat_level': threat_name,
                'simulation_count': self.config.num_simulations,
                'version': '3.0'
            },
            'configuration': {
                'area': {
                    'width_m': self.config.area_width,
                    'height_m': self.config.area_height,
                    'max_depth_m': self.config.max_depth
                },
                'total_mines': self.config.threat_level.value[0],
                'target_risk': self.config.threat_level.value[1],
                'deployment': {
                    'linear_ratio': self.config.linear_density,
                    'random_ratio': self.config.random_density,
                    'core_route_width_m': self.config.core_route_width
                },
                'mine_parameters': {
                    'surface_depth_range_m': self.config.surface_mine_depth_range,
                    'surface_spacing_m': self.config.surface_mine_spacing,
                    'subsurface_depth_range_m': self.config.subsurface_mine_depth_range,
                    'subsurface_spacing_m': self.config.subsurface_mine_spacing,
                    'mine_radius_m': self.config.mine_radius
                }
            },
            'results': {
                'surface_vessel': {
                    'probabilities': {
                        'mine_hit': round(stats['surface_vessel']['mine_hit_prob'], 4),
                        'net_hit': round(stats['surface_vessel']['net_hit_prob'], 4),
                        'both_hit': round(stats['surface_vessel']['both_hit_prob'], 4),
                        'any_hit': round(stats['surface_vessel']['any_hit_prob'], 4),
                        'safe_passage': round(stats['surface_vessel']['safe_prob'], 4)
                    },
                    'counts': stats['surface_vessel']['counts']
                },
                'submarine': {
                    'probabilities': {
                        'mine_hit': round(stats['submarine']['mine_hit_prob'], 4),
                        'net_hit': round(stats['submarine']['net_hit_prob'], 4),
                        'both_hit': round(stats['submarine']['both_hit_prob'], 4),
                        'any_hit': round(stats['submarine']['any_hit_prob'], 4),
                        'safe_passage': round(stats['submarine']['safe_prob'], 4)
                    },
                    'counts': stats['submarine']['counts']
                }
            }
        }

        # 시나리오 결과 추가
        if scenario_results:
            export_data['route_scenarios'] = {}
            for scenario_name, result in scenario_results.items():
                export_data['route_scenarios'][scenario_name] = {
                    'probabilities': {
                        'mine_hit': round(result['mine_hit_prob'], 4),
                        'net_hit': round(result['net_hit_prob'], 4),
                        'both_hit': round(result['both_hit_prob'], 4),
                        'any_hit': round(result['any_hit_prob'], 4),
                        'safe_passage': round(result['safe_prob'], 4)
                    },
                    'counts': result['counts']
                }

        filename = f'{output_dir}/results_{threat_name.lower()}.json'
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(export_data, f, indent=2, ensure_ascii=False)

        print(f"✓ Saved: results_{threat_name.lower()}.json")

    def visualize_tactical_deployment(self,
                                     surface_start: Tuple[float, float],
                                     surface_end: Tuple[float, float],
                                     sub_start: Tuple[float, float, float],
                                     sub_end: Tuple[float, float, float],
                                     save_path: str = None):
        """전술적 기뢰 배치 시각화"""
        self.generate_tactical_mines(surface_start, surface_end, seed=42)

        fig = plt.figure(figsize=(20, 10))

        # === 3D 뷰 ===
        ax1 = fig.add_subplot(121, projection='3d')
        ax1.set_facecolor('#0a1929')

        # 핵심 항로 표시
        route_x = [surface_start[0], surface_end[0]]
        route_y = [surface_start[1], surface_end[1]]
        route_z = [0, 0]
        ax1.plot(route_x, route_y, route_z, 'y-', linewidth=6,
                alpha=0.5, label='Core Route', zorder=1)

        # 부유 기뢰
        linear_surf = [m for m in self.surface_mines if m.placement_type == "linear"]
        random_surf = [m for m in self.surface_mines if m.placement_type == "random"]

        for mine in linear_surf[:20]:
            ax1.scatter(mine.x, mine.y, mine.z, c='#ff0000', s=150,
                       marker='*', edgecolors='yellow', linewidths=2, alpha=0.8)

        for mine in random_surf[:20]:
            ax1.scatter(mine.x, mine.y, mine.z, c='#ff6666', s=80,
                       marker='*', edgecolors='white', linewidths=1, alpha=0.6)

        # 계류 기뢰
        linear_moored = [m for m in self.moored_mines if m.placement_type == "linear"]
        random_moored = [m for m in self.moored_mines if m.placement_type == "random"]

        for mine in linear_moored[:20]:
            ax1.scatter(mine.x, mine.y, mine.z, c='#ff9500', s=120,
                       marker='o', edgecolors='yellow', linewidths=2, alpha=0.8)
            ax1.plot([mine.x, mine.x], [mine.y, mine.y], [0, mine.z],
                    'orange', alpha=0.4, linewidth=1, linestyle='--')

        for mine in random_moored[:20]:
            ax1.scatter(mine.x, mine.y, mine.z, c='#ffbb66', s=60,
                       marker='o', edgecolors='white', linewidths=1, alpha=0.6)

        # 침저 기뢰
        linear_bottom = [m for m in self.bottom_mines if m.placement_type == "linear"]
        random_bottom = [m for m in self.bottom_mines if m.placement_type == "random"]

        bottom_x_l = [m.x for m in linear_bottom[:20]]
        bottom_y_l = [m.y for m in linear_bottom[:20]]
        bottom_z_l = [m.z for m in linear_bottom[:20]]
        ax1.scatter(bottom_x_l, bottom_y_l, bottom_z_l, c='#8b4513',
                   s=150, marker='^', edgecolors='yellow', linewidths=2, alpha=0.8)

        bottom_x_r = [m.x for m in random_bottom[:20]]
        bottom_y_r = [m.y for m in random_bottom[:20]]
        bottom_z_r = [m.z for m in random_bottom[:20]]
        ax1.scatter(bottom_x_r, bottom_y_r, bottom_z_r, c='#aa6633',
                   s=80, marker='^', edgecolors='white', linewidths=1, alpha=0.6)

        # 닻자망
        for net in self.nets[:10]:
            verts = [
                [net.x1, net.y1, net.z_top],
                [net.x2, net.y2, net.z_top],
                [net.x2, net.y2, net.z_bottom],
                [net.x1, net.y1, net.z_bottom]
            ]
            poly = Poly3DCollection([verts], alpha=0.15, facecolor='#0066ff',
                                   edgecolor='#0044cc', linewidths=1.5)
            ax1.add_collection3d(poly)

        # 경로
        surface_path = self.generate_path_2d(surface_start, surface_end)
        sub_path = self.generate_path_3d(sub_start, sub_end)

        mine_hit_s, net_hit_s = self.check_surface_vessel_safety(surface_path)
        mine_hit_sub, net_hit_sub = self.check_submarine_safety(sub_path)

        color_surface = '#ff4444' if (mine_hit_s or net_hit_s) else '#00ff88'
        color_sub = '#ff4444' if (mine_hit_sub or net_hit_sub) else '#00ddff'

        ax1.plot(surface_path[:, 0], surface_path[:, 1], surface_path[:, 2],
                color=color_surface, linewidth=4, label='Surface Vessel',
                alpha=0.9, zorder=100)

        ax1.plot(sub_path[:, 0], sub_path[:, 1], sub_path[:, 2],
                color=color_sub, linewidth=4, label='Submarine',
                alpha=0.9, linestyle='--', zorder=100)

        # 시작/끝점
        ax1.scatter(*surface_start, 0, c='#00ff88', s=400, marker='o',
                   edgecolors='white', linewidths=3, zorder=200)
        ax1.scatter(*surface_end, 0, c='#00ff88', s=400, marker='s',
                   edgecolors='white', linewidths=3, zorder=200)
        ax1.scatter(*sub_start, c='#00ddff', s=400, marker='o',
                   edgecolors='white', linewidths=3, zorder=200)
        ax1.scatter(*sub_end, c='#00ddff', s=400, marker='s',
                   edgecolors='white', linewidths=3, zorder=200)

        ax1.set_xlabel('East (m)', fontsize=12, fontweight='bold', color='white')
        ax1.set_ylabel('North (m)', fontsize=12, fontweight='bold', color='white')
        ax1.set_zlabel('Depth (m)', fontsize=12, fontweight='bold', color='white')
        ax1.set_title('Tactical Mine Deployment - 3D View',
                     fontsize=15, fontweight='bold', pad=20, color='white')
        ax1.legend(loc='upper left', fontsize=10, framealpha=0.9,
                  facecolor='#1e1e1e', edgecolor='white', labelcolor='white')

        ax1.set_xlim(0, self.config.area_width)
        ax1.set_ylim(0, self.config.area_height)
        ax1.set_zlim(self.config.max_depth, -50)
        ax1.view_init(elev=25, azim=45)

        ax1.xaxis.pane.fill = False
        ax1.yaxis.pane.fill = False
        ax1.zaxis.pane.fill = False
        ax1.xaxis.pane.set_edgecolor('white')
        ax1.yaxis.pane.set_edgecolor('white')
        ax1.zaxis.pane.set_edgecolor('white')
        ax1.grid(True, alpha=0.2, color='white')
        ax1.tick_params(colors='white')

        # === 2D 탑뷰 ===
        ax2 = fig.add_subplot(122)
        ax2.set_facecolor('#f0f4f8')

        # 핵심 항로 영역
        direction, perpendicular = self._calculate_core_route(surface_start, surface_end)
        route_width = self.config.core_route_width

        corners = [
            [surface_start[0] - perpendicular[0] * route_width/2,
             surface_start[1] - perpendicular[1] * route_width/2],
            [surface_end[0] - perpendicular[0] * route_width/2,
             surface_end[1] - perpendicular[1] * route_width/2],
            [surface_end[0] + perpendicular[0] * route_width/2,
             surface_end[1] + perpendicular[1] * route_width/2],
            [surface_start[0] + perpendicular[0] * route_width/2,
             surface_start[1] + perpendicular[1] * route_width/2]
        ]
        route_poly = mpatches.Polygon(corners, color='yellow', alpha=0.15,
                                     edgecolor='yellow', linewidth=3, linestyle='--',
                                     label='Core Route Zone')
        ax2.add_patch(route_poly)

        # 기뢰 표시
        linear_x = [m.x for m in linear_surf]
        linear_y = [m.y for m in linear_surf]
        ax2.scatter(linear_x, linear_y, c='#ff0000', s=150, marker='*',
                   edgecolors='yellow', linewidths=2, alpha=0.8,
                   label='Surface Mines (Linear)', zorder=10)

        random_x = [m.x for m in random_surf]
        random_y = [m.y for m in random_surf]
        ax2.scatter(random_x, random_y, c='#ff6666', s=60, marker='*',
                   edgecolors='white', linewidths=1, alpha=0.6,
                   label='Surface Mines (Random)', zorder=9)

        linear_x = [m.x for m in linear_moored]
        linear_y = [m.y for m in linear_moored]
        ax2.scatter(linear_x, linear_y, c='#ff9500', s=120, marker='o',
                   edgecolors='yellow', linewidths=2, alpha=0.8,
                   label='Moored Mines (Linear)', zorder=8)

        random_x = [m.x for m in random_moored]
        random_y = [m.y for m in random_moored]
        ax2.scatter(random_x, random_y, c='#ffbb66', s=50, marker='o',
                   edgecolors='white', linewidths=1, alpha=0.6,
                   label='Moored Mines (Random)', zorder=7)

        linear_x = [m.x for m in linear_bottom]
        linear_y = [m.y for m in linear_bottom]
        ax2.scatter(linear_x, linear_y, c='#8b4513', s=120, marker='^',
                   edgecolors='yellow', linewidths=2, alpha=0.8,
                   label='Bottom Mines (Linear)', zorder=6)

        random_x = [m.x for m in random_bottom]
        random_y = [m.y for m in random_bottom]
        ax2.scatter(random_x, random_y, c='#aa6633', s=50, marker='^',
                   edgecolors='white', linewidths=1, alpha=0.6,
                   label='Bottom Mines (Random)', zorder=5)

        # 닻자망
        for net in self.nets:
            ax2.plot([net.x1, net.x2], [net.y1, net.y2],
                    'b-', linewidth=2.5, alpha=0.5)

        # 경로
        ax2.plot(surface_path[:, 0], surface_path[:, 1],
                color=color_surface, linewidth=4, label='Surface Vessel Path',
                alpha=0.9, zorder=15)
        ax2.plot(sub_path[:, 0], sub_path[:, 1],
                color=color_sub, linewidth=4, label='Submarine Path',
                alpha=0.9, linestyle='--', zorder=15)

        # 시작/끝점
        ax2.scatter(*surface_start, c='#00ff88', s=400, marker='o',
                   edgecolors='black', linewidths=3, zorder=20)
        ax2.scatter(*surface_end, c='#00ff88', s=400, marker='s',
                   edgecolors='black', linewidths=3, zorder=20)

        ax2.set_xlim(-200, self.config.area_width + 200)
        ax2.set_ylim(-200, self.config.area_height + 200)
        ax2.set_aspect('equal')
        ax2.set_xlabel('East (m)', fontsize=12, fontweight='bold')
        ax2.set_ylabel('North (m)', fontsize=12, fontweight='bold')
        ax2.set_title('Tactical Mine Deployment - Top View',
                     fontsize=15, fontweight='bold', pad=15)
        ax2.legend(loc='upper right', fontsize=9, framealpha=0.95, ncol=2)
        ax2.grid(True, alpha=0.3, linestyle='--')

        # 정보 박스
        threat_name = self.config.threat_level.name
        threat_mines, threat_rate = self.config.threat_level.value

        info_text = f"""TACTICAL DEPLOYMENT

Threat Level: {threat_name}
Target Risk: {threat_rate*100:.0f}%
Total Mines: {threat_mines}

Deployment:
- Surface: {len(self.surface_mines)}
  - Linear: {len(linear_surf)}
  - Random: {len(random_surf)}

- Moored: {len(self.moored_mines)}
  - Linear: {len(linear_moored)}
  - Random: {len(random_moored)}

- Bottom: {len(self.bottom_mines)}
  - Linear: {len(linear_bottom)}
  - Random: {len(random_bottom)}

- Nets: {len(self.nets)}

Route: {self.config.core_route_width}m wide"""

        bbox_props = dict(boxstyle='round,pad=0.8', facecolor='#fffef7',
                         edgecolor='#333', linewidth=2, alpha=0.95)
        ax2.text(0.02, 0.98, info_text, transform=ax2.transAxes,
                fontsize=9, verticalalignment='top', family='monospace',
                bbox=bbox_props, fontweight='bold')

        plt.suptitle(f'Tactical Mine Deployment - {threat_name} Threat Level\n\n',
                    fontsize=22, fontweight='bold', y=0.98)
        plt.tight_layout()

        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')

        return fig

    def visualize_route_scenarios(self,
                                  sub_start: Tuple[float, float, float],
                                  sub_end: Tuple[float, float, float],
                                  output_dir: str):
        """경로 시나리오 비교 시각화"""
        if not self.scenario_results:
            print("No scenario results available. Run scenario comparison first.")
            return

        self.generate_tactical_mines((sub_start[0], sub_start[1]),
                                    (sub_end[0], sub_end[1]), seed=42)

        fig = plt.figure(figsize=(20, 12))
        gs = fig.add_gridspec(2, 2, hspace=0.3, wspace=0.3)

        scenarios = list(RouteScenario)
        colors = ['#3498db', '#e74c3c', '#f39c12', '#9b59b6']

        # === 4개 시나리오 3D 시각화 ===
        for idx, (scenario, color) in enumerate(zip(scenarios, colors)):
            ax = fig.add_subplot(gs[idx // 2, idx % 2], projection='3d')
            ax.set_facecolor('#f0f4f8')

            # 경로 생성
            path = self.generate_scenario_path(scenario, sub_start, sub_end)
            mine_hit, net_hit = self.check_submarine_safety(path)

            path_color = '#ff4444' if (mine_hit or net_hit) else color

            # 기뢰 표시 (간소화)
            for mine in self.surface_mines[:10]:
                ax.scatter(mine.x, mine.y, mine.z, c='red', s=50, alpha=0.3, marker='*')

            for mine in self.moored_mines[:10]:
                ax.scatter(mine.x, mine.y, mine.z, c='orange', s=40, alpha=0.3, marker='o')

            for mine in self.bottom_mines[:10]:
                ax.scatter(mine.x, mine.y, mine.z, c='brown', s=40, alpha=0.3, marker='^')

            # 경로
            ax.plot(path[:, 0], path[:, 1], path[:, 2],
                   color=path_color, linewidth=4, alpha=0.9, label=scenario.value)

            # 시작/끝점
            ax.scatter(*sub_start, c='green', s=300, marker='o',
                      edgecolors='black', linewidths=2, zorder=100)
            ax.scatter(*sub_end, c='green', s=300, marker='s',
                      edgecolors='black', linewidths=2, zorder=100)

            ax.set_xlabel('East (m)', fontsize=10)
            ax.set_ylabel('North (m)', fontsize=10)
            ax.set_zlabel('Depth (m)', fontsize=10)

            # 통계 표시
            result = self.scenario_results[scenario.value]
            title = f"{scenario.value}\nSafe: {result['safe_prob']*100:.1f}% | Risk: {result['any_hit_prob']*100:.1f}%"
            ax.set_title(title, fontsize=12, fontweight='bold', pad=10)

            ax.set_xlim(0, self.config.area_width)
            ax.set_ylim(0, self.config.area_height)
            ax.set_zlim(self.config.max_depth, -50)
            ax.view_init(elev=20, azim=45)
            ax.grid(True, alpha=0.3)

        plt.suptitle(f'Route Scenario Comparison - {self.config.threat_level.name} Threat',
                    fontsize=22, fontweight='bold', y=0.98)

        save_path = f'{output_dir}/route_scenarios_{self.config.threat_level.name.lower()}.png'
        plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()

        print(f"✓ Saved: route_scenarios_{self.config.threat_level.name.lower()}.png")


def create_comparison_dashboard(all_results: Dict, output_dir: str):
    """종합 비교 대시보드 생성"""
    fig, axes = plt.subplots(2, 2, figsize=(18, 14))

    threat_names = ['MODERATE', 'HIGH', 'CRITICAL']
    mines = [150, 300, 450]

    # 1. 위험도 비교 막대 그래프
    ax1 = axes[0, 0]
    surface_risks = [all_results[t]['surface_vessel']['any_hit_prob']*100 for t in threat_names]
    sub_risks = [all_results[t]['submarine']['any_hit_prob']*100 for t in threat_names]

    x = np.arange(len(threat_names))
    width = 0.35

    bars1 = ax1.bar(x - width/2, surface_risks, width, label='Surface Vessel',
                   color='#3498db', alpha=0.8, edgecolor='black', linewidth=2)
    bars2 = ax1.bar(x + width/2, sub_risks, width, label='Submarine',
                   color='#e74c3c', alpha=0.8, edgecolor='black', linewidth=2)

    # 값 표시
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            ax1.text(bar.get_x() + bar.get_width()/2., height + 1,
                    f'{height:.1f}%', ha='center', va='bottom',
                    fontsize=11, fontweight='bold')

    ax1.set_ylabel('Risk Probability (%)', fontsize=13, fontweight='bold')
    ax1.set_title('Risk Level Comparison by Threat Level', fontsize=14, fontweight='bold')
    ax1.set_xticks(x)
    ax1.set_xticklabels(threat_names)
    ax1.legend(fontsize=11)
    ax1.grid(True, alpha=0.3, axis='y')
    ax1.set_ylim(0, max(surface_risks + sub_risks) * 1.15)

    # 2. 생존율 트렌드
    ax2 = axes[0, 1]
    surface_survival = [all_results[t]['surface_vessel']['safe_prob']*100 for t in threat_names]
    sub_survival = [all_results[t]['submarine']['safe_prob']*100 for t in threat_names]

    ax2.plot(mines, surface_survival, 'o-', linewidth=3, markersize=12,
            label='Surface Vessel', color='#3498db', markeredgecolor='black', markeredgewidth=2)
    ax2.plot(mines, sub_survival, 's-', linewidth=3, markersize=12,
            label='Submarine', color='#e74c3c', markeredgecolor='black', markeredgewidth=2)

    # 값 표시
    for i, (m, ss, subs) in enumerate(zip(mines, surface_survival, sub_survival)):
        ax2.text(m, ss + 2, f'{ss:.1f}%', ha='center', fontsize=10, fontweight='bold')
        ax2.text(m, subs - 4, f'{subs:.1f}%', ha='center', fontsize=10, fontweight='bold')

    ax2.set_xlabel('Number of Mines', fontsize=13, fontweight='bold')
    ax2.set_ylabel('Survival Rate (%)', fontsize=13, fontweight='bold')
    ax2.set_title('Survival Rate vs Mine Density', fontsize=14, fontweight='bold')
    ax2.legend(fontsize=11)
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim(0, 105)

    # 3. 잠수함 우위도
    ax3 = axes[1, 0]
    advantage = [sub_survival[i] - surface_survival[i] for i in range(3)]
    colors_adv = ['#27ae60' if a > 0 else '#e74c3c' for a in advantage]

    bars3 = ax3.bar(threat_names, advantage, color=colors_adv, alpha=0.7,
                   edgecolor='black', linewidth=2.5)

    for bar, adv in zip(bars3, advantage):
        height = bar.get_height()
        ax3.text(bar.get_x() + bar.get_width()/2., height + (1 if height > 0 else -3),
                f'{adv:+.1f}%', ha='center', va='bottom' if height > 0 else 'top',
                fontsize=12, fontweight='bold')

    ax3.axhline(y=0, color='black', linestyle='--', linewidth=2)
    ax3.set_ylabel('Submarine Advantage (%)', fontsize=13, fontweight='bold')
    ax3.set_title('Submarine Safety Advantage Over Surface', fontsize=14, fontweight='bold')
    ax3.grid(True, alpha=0.3, axis='y')

    # 4. 통계 테이블
    ax4 = axes[1, 1]
    ax4.axis('off')

    table_data = []
    for i, threat in enumerate(threat_names):
        surf_risk = all_results[threat]['surface_vessel']['any_hit_prob']*100
        sub_risk = all_results[threat]['submarine']['any_hit_prob']*100
        surf_safe = all_results[threat]['surface_vessel']['safe_prob']*100
        sub_safe = all_results[threat]['submarine']['safe_prob']*100

        table_data.append([
            threat,
            f"{mines[i]}",
            f"{surf_risk:.1f}%",
            f"{sub_risk:.1f}%",
            f"{surf_safe:.1f}%",
            f"{sub_safe:.1f}%",
            f"{advantage[i]:+.1f}%"
        ])

    table = ax4.table(cellText=table_data,
                     colLabels=['Threat\nLevel', 'Mines', 'Surface\nRisk', 'Sub\nRisk',
                               'Surface\nSafe', 'Sub\nSafe', 'Sub\nAdvantage'],
                     cellLoc='center',
                     loc='center',
                     bbox=[0, 0.1, 1, 0.8])

    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 2.5)

    # 헤더 스타일
    for j in range(7):
        cell = table[(0, j)]
        cell.set_facecolor('#2c3e50')
        cell.set_text_props(weight='bold', color='white', fontsize=11)

    # 데이터 행 스타일
    for i in range(1, len(table_data) + 1):
        for j in range(7):
            cell = table[(i, j)]
            cell.set_facecolor('#ecf0f1' if i % 2 == 0 else 'white')
            cell.set_text_props(fontsize=10)

            # 마지막 열 (우위도) 색상
            if j == 6:
                value = float(table_data[i-1][j].replace('%', '').replace('+', ''))
                if value > 0:
                    cell.set_facecolor('#d5f4e6')
                else:
                    cell.set_facecolor('#fadbd8')

    ax4.text(0.5, 0.95, 'Comprehensive Statistics Summary',
            ha='center', va='top', transform=ax4.transAxes,
            fontsize=14, fontweight='bold')

    plt.suptitle('Tactical Mine Warfare Simulation - Comprehensive Analysis Dashboard\n\n',
                fontsize=22, fontweight='bold', y=0.98)
    plt.tight_layout()

    save_path = f'{output_dir}/comparison_dashboard.png'
    plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
    plt.close()

    print(f"\n✓ Saved: comparison_dashboard.png")


# ===== 메인 실행 =====
def main():
    print("="*80)
    print("ADVANCED TACTICAL MINE DEPLOYMENT SIMULATION".center(80))
    print("="*80)

    output_dir = './output'
    os.makedirs(output_dir, exist_ok=True)

    # 세 가지 위협 수준으로 테스트
    threat_levels = [ThreatLevel.MODERATE, ThreatLevel.HIGH, ThreatLevel.CRITICAL]

    surface_start = (1000, 1000)
    surface_end = (9000, 9000)
    sub_start = (1000, 1000, 100)
    sub_end = (9000, 9000, 200)

    all_results = {}

    for threat in threat_levels:
        print(f"\n{'='*80}")
        print(f"THREAT LEVEL: {threat.name}".center(80))
        print(f"Target Mines: {threat.value[0]}, Target Risk: {threat.value[1]*100:.0f}%".center(80))
        print(f"{'='*80}\n")

        config = TacticalMineConfig(
            threat_level=threat,
        )

        sim = TacticalMineSimulation(config)

        # 기본 시뮬레이션 실행
        print(f"Running simulations...\n")
        stats = sim.run_simulation(surface_start, surface_end, sub_start, sub_end, verbose=True)

        all_results[threat.name] = stats

        # 결과 출력
        print(f"\n{'RESULTS':^80}")
        print("─" * 80)

        for vtype, title in [('surface_vessel', 'Surface Vessel'), ('submarine', 'Submarine')]:
            s = stats[vtype]
            print(f"{title}:")
            print(f"  Total Risk:   {s['any_hit_prob']*100:6.2f}%")
            print(f"  Safe Transit: {s['safe_prob']*100:6.2f}%")

        # 경로 시나리오 비교 (잠수함만)
        scenario_results = sim.run_scenario_comparison(sub_start, sub_end, num_iterations=100)

        # 시각화
        print(f"\nGenerating visualizations for {threat.name}...")

        # 1. 전술 배치도
        fig = sim.visualize_tactical_deployment(
            surface_start, surface_end, sub_start, sub_end,
            save_path=f'{output_dir}/tactical_{threat.name.lower()}.png'
        )
        plt.close(fig)
        print(f"✓ Saved: tactical_{threat.name.lower()}.png")

        # 2. 경로 시나리오 비교
        sim.visualize_route_scenarios(sub_start, sub_end, output_dir)

        # 3. JSON 결과 저장
        sim.export_results_json(stats, threat.name, output_dir, scenario_results)

    # 종합 비교 대시보드
    print("\n" + "="*80)
    print("Generating comprehensive comparison dashboard...")
    create_comparison_dashboard(all_results, output_dir)

    # 비교 요약 출력
    print("\n" + "="*80)
    print("COMPARISON SUMMARY".center(80))
    print("="*80)
    print(f"\n{'Threat Level':<15} {'Mines':<10} {'Surface Risk':<15} {'Sub Risk':<15} {'Sub Advantage':<15} {'Target':<10}")
    print("─" * 80)

    for threat in threat_levels:
        name = threat.name
        mines, target = threat.value
        surf_risk = all_results[name]['surface_vessel']['any_hit_prob'] * 100
        sub_risk = all_results[name]['submarine']['any_hit_prob'] * 100
        advantage = all_results[name]['submarine']['safe_prob'] * 100 - all_results[name]['surface_vessel']['safe_prob'] * 100

        print(f"{name:<15} {mines:<10} {surf_risk:>6.2f}%{'':<8} {sub_risk:>6.2f}%{'':<8} {advantage:>+6.2f}%{'':<8} {target*100:.0f}%")

    print("\n" + "="*80)
    print("✓ ALL SIMULATIONS COMPLETED!".center(80))
    print("="*80)
    print("\nGenerated Files:")
    print("  • tactical_moderate.png       - MODERATE threat deployment")
    print("  • tactical_high.png           - HIGH threat deployment")
    print("  • tactical_critical.png       - CRITICAL threat deployment")
    print("  • route_scenarios_*.png       - Route scenario comparisons (3 files)")
    print("  • comparison_dashboard.png    - Comprehensive analysis dashboard")
    print("  • results_*.json              - Detailed statistics (3 files)")
    print("="*80)

if __name__ == "__main__":
    main()

                  ADVANCED TACTICAL MINE DEPLOYMENT SIMULATION                  

                             THREAT LEVEL: MODERATE                             
                       Target Mines: 73, Target Risk: 50%                       

Running simulations...

Progress: 4.8% (12/250)
Progress: 9.6% (24/250)
Progress: 14.4% (36/250)
Progress: 19.2% (48/250)
Progress: 24.0% (60/250)
Progress: 28.8% (72/250)
Progress: 33.6% (84/250)
Progress: 38.4% (96/250)
Progress: 43.2% (108/250)
Progress: 48.0% (120/250)
Progress: 52.8% (132/250)
Progress: 57.6% (144/250)
Progress: 62.4% (156/250)
Progress: 67.2% (168/250)
Progress: 72.0% (180/250)
Progress: 76.8% (192/250)
Progress: 81.6% (204/250)
Progress: 86.4% (216/250)
Progress: 91.2% (228/250)
Progress: 96.0% (240/250)

                                    RESULTS                                     
────────────────────────────────────────────────────────────────────────────────
Surface Vessel:
  Total Risk:    17.60%
  Safe Transit:  8

/tmp/ipython-input-1845095888.py:824: UserWarning: Setting the 'color' property will override the edgecolor or facecolor properties.
  route_poly = mpatches.Polygon(corners, color='yellow', alpha=0.15,


✓ Saved: tactical_moderate.png
✓ Saved: route_scenarios_moderate.png
✓ Saved: results_moderate.json

                               THREAT LEVEL: HIGH                               
                      Target Mines: 145, Target Risk: 75%                       

Running simulations...

Progress: 4.8% (12/250)
Progress: 9.6% (24/250)
Progress: 14.4% (36/250)
Progress: 19.2% (48/250)
Progress: 24.0% (60/250)
Progress: 28.8% (72/250)
Progress: 33.6% (84/250)
Progress: 38.4% (96/250)
Progress: 43.2% (108/250)
Progress: 48.0% (120/250)
Progress: 52.8% (132/250)
Progress: 57.6% (144/250)
Progress: 62.4% (156/250)
Progress: 67.2% (168/250)
Progress: 72.0% (180/250)
Progress: 76.8% (192/250)
Progress: 81.6% (204/250)
Progress: 86.4% (216/250)
Progress: 91.2% (228/250)
Progress: 96.0% (240/250)

                                    RESULTS                                     
────────────────────────────────────────────────────────────────────────────────
Surface Vessel:
  Total Risk:    30.00%